In [232]:
import json
from pprint import pprint

def GetRequiredResumeDetails(ScrapedText):
    value = newText[0]
    extractedJSONStr = value.encode('utf8').decode('unicode_escape')
    jsonObj = json.loads(extractedJSONStr)
    #pprint(jsonObj['resumeModel'])
    requiredTags = ['fullName','headline','education','additionalInformation','workExperience', 'headline','skills']
    educationRequiredString = []
    skillsRequiredString = []
    companyDetails = []
    counter = 0
    resumeText = []
    totalDataDict = {}
    for j in jsonObj['resumeModel']:
        if(j in requiredTags):
            if(j == 'additionalInformation'):
                reqText = re.sub(r'\s+',' ',jsonObj['resumeModel'][j])
                #print (reqText)
                resumeText.append(reqText)
            if(j == 'education'):
                for x in range(0,len(jsonObj['resumeModel'][j])):
                    reqStr = jsonObj['resumeModel'][j][x]['dateRange']+" "+jsonObj['resumeModel'][j][x]['degree']+" "+jsonObj['resumeModel'][j][x]['field']+" "+jsonObj['resumeModel'][j][x]['location']+" "+jsonObj['resumeModel'][j][x]['university']
                    educationRequiredString.append(reqStr)
                educationDetails = "Education Details \n"
                educationDetails += "\n".join(educationRequiredString)
                resumeText.append(educationDetails)
            if(j == 'fullName'):
                #print (jsonObj['resumeModel'][j])
                resumeText.append(jsonObj['resumeModel'][j])

            if(j == 'headline'):
                #print (jsonObj['resumeModel'][j])
                resumeText.append(jsonObj['resumeModel'][j])

            if(j == 'skills'):
                #print  (len(jsonObj['resumeModel'][j]))
                for a in range(0,len(jsonObj['resumeModel'][j])):
                    monthsOfExperience = ""
                    if(jsonObj['resumeModel'][j][a]['monthsOfExperience'] == 0):
                        monthsOfExperience = "Less than 1 year"
                    else:
                        monthsOfExperience = jsonObj['resumeModel'][j][a]['monthsOfExperience']
                    string = jsonObj['resumeModel'][j][a]['skill']+"- Exprience - "+str(monthsOfExperience)+" months"
                    skillsRequiredString.append(string)

                skillDetails = "Skill Details \n"
                skillDetails += "\n".join(skillsRequiredString)
                #print (skillDetails)
                resumeText.append(skillDetails)

            if(j == "workExperience"):
                for b in range(0,len(jsonObj['resumeModel'][j])):
                    for tag in ['company','description']:
                        wkString = tag+" - "+jsonObj['resumeModel'][j][b][tag]
                        companyDetails.append(wkString)

                compDetails = "Company Details \n"
                compDetails += "\n".join(companyDetails)
                #print (compDetails)
                resumeText.append(compDetails)
    return resumeText

In [257]:
from bs4 import BeautifulSoup
import urllib3
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

http = urllib3.PoolManager()

mainDict = {
'Data Science' : ["https://resumes.indeed.com/resume/19936d88d452cba8?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/fc69b971ab0f7feb?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/e7a80d47fd3b870a?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/a0e2efc28ffe7076?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/4416a22a685fe75f?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/a7ac5303fd482fbb?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/01303e67a9a9f907?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/adf5c0dcf7fb32c1?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/0dcb97d5e222ff21?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt",
   "https://resumes.indeed.com/resume/e8aaa1f939a0a899?s=l%3D%26q%3DData%2520Science%26searchFields%3Djt"],
'HR' : ["https://resumes.indeed.com/resume/5b3f2a5a5f830e7d?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/8211f264641ea39c?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/fbb79851e4d6c8e7?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/21d29b355eca455d?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/21d29b355eca455d?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/87f089302484e5aa?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/fb84374d438cc1f8?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/27ca206713ab314a?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/ecc72d286fd39dc3?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/4273f1b1a23eb6df?s=l%3D%26q%3DHR%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/2711c1cd24df34a8?s=l%3D%26q%3DHR%26searchFields%3Djt"],
'Advocate' : ["https://resumes.indeed.com/resume/497ede84a446ce67?s=l%3D%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/183065eed2881757?s=l%3D%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/38ee4808c0b7774c?s=l%3D%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/c53da6d49f7d9c8b?s=l%3D%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/a777948727a94e37?s=l%3D%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/db5e3efd47ef4ff1?s=l%3D%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/e50429299927d330?s=l%3DMaharashtra%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/926f38ba69190e63?s=l%3DMaharashtra%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/9c98609f16f88f1d?s=l%3DMaharashtra%26q%3DAdvocate%26searchFields%3Djt",
             "https://resumes.indeed.com/resume/7307abd462e73736?s=l%3DMaharashtra%26q%3DAdvocate%26searchFields%3Djt"],
'Arts':["https://resumes.indeed.com/resume/9adceec7573356f7?s=l%3DMaharashtra%26q%3DArts%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/d194fa2332631d55?s=l%3DMaharashtra%26q%3DArts%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/415f154981e0d325?s=l%3DMaharashtra%26q%3DArts%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/6f240797fb98264f?s=l%3DMaharashtra%26q%3DArts%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/eb02fae68d77084c?s=l%3DMaharashtra%26q%3DArts%26searchFields%3Djt",
       "https://resumes.indeed.com/resume/d0a2e5c9c2466955?s=l%3DMaharashtra%26q%3DArts%26searchFields%3Djt"
       ],
'Web Designing':["https://resumes.indeed.com/resume/d84e64ef45cf407c?s=l%3DMaharashtra%26q%3DWeb%2520Designing%26searchFields%3Djt",
     "https://resumes.indeed.com/resume/75278f7b4cd5d441?s=l%3DMaharashtra%26q%3DWeb%2520Designing%26searchFields%3Djt",
     "https://resumes.indeed.com/resume/d84e64ef45cf407c?s=l%3DMaharashtra%26q%3DWeb%2520Designing%26searchFields%3Djt",
     "https://resumes.indeed.com/resume/e7f4f09941f0bab3?s=l%3DMaharashtra%26q%3DWeb%2520Designing%26searchFields%3Djt",
     "https://resumes.indeed.com/resume/7c7f31af8bb98278?s=l%3DMaharashtra%26q%3DWeb%2520Designing%26searchFields%3Djt"
                  ],
'Mechanical Engineer':
    ["https://resumes.indeed.com/resume/ed76e70d01c5cc55?s=l%3DMaharashtra%26q%3Dmechanical%2520engineering%26searchFields%3D",
     "https://resumes.indeed.com/resume/a889bf7b96e62864?s=l%3DMaharashtra%26q%3Dmechanical%2520engineering%26searchFields%3D",
     "https://resumes.indeed.com/resume/0bbdbc0f68b75575?s=l%3DMaharashtra%26q%3Dmechanical%2520engineering%26searchFields%3D",
     "https://resumes.indeed.com/resume/6756c01cb6391e15?s=l%3DMaharashtra%26q%3Dmechanical%2520engineering%26searchFields%3D",
     "https://resumes.indeed.com/resume/2871c94629cb3c8c?s=l%3DMaharashtra%26q%3Dmechanical%2520engineering%26searchFields%3D"
                      ],
"Sales":["https://resumes.indeed.com/resume/767b41f8db4b5ac4?s=l%3DMaharashtra%26q%3Dsales%2520manager%26searchFields%3Djt",
        "https://resumes.indeed.com/resume/5aa9e638d5c222c8?s=l%3DMaharashtra%26q%3Dsales%2520manager%26searchFields%3Djt",
        "https://resumes.indeed.com/resume/3511266d84655935?s=l%3DMaharashtra%26q%3Dsales%2520manager%26searchFields%3Djt",
        "https://resumes.indeed.com/resume/78ee58ff06d87647?s=l%3DMaharashtra%26q%3Dsales%2520manager%26searchFields%3Djt",
        "https://resumes.indeed.com/resume/0272499c2dfb4758?s=l%3DMaharashtra%26q%3Dsales%2520manager%26searchFields%3Djt"],
"Health and fitness":
    ["https://resumes.indeed.com/resume/e279e978534e0137?s=l%3DMaharashtra%26q%3Dhealth%2520and%2520fitness%26searchFields%3D",
    "https://resumes.indeed.com/resume/aa37c969a5ae8e31?s=l%3DMaharashtra%26q%3DHealth%2520and%2520fitness%26searchFields%3D",
    "https://resumes.indeed.com/resume/abf159c7880e8ecb?s=l%3DMaharashtra%26q%3DHealth%2520and%2520fitness%26searchFields%3D",
    "https://resumes.indeed.com/resume/e45f0b0debe0065a?s=l%3DMaharashtra%26q%3DHealth%2520and%2520fitness%26searchFields%3D",
    "https://resumes.indeed.com/resume/b052cbc7373be565?s=l%3DMaharashtra%26q%3DHealth%2520and%2520fitness%26searchFields%3D",
    "https://resumes.indeed.com/resume/5702cb0c630caa96?s=l%3DMaharashtra%26q%3DHealth%2520and%2520fitness%26searchFields%3D"],
"Civil Engineer":
    ["https://resumes.indeed.com/resume/d72ad00024217885?s=l%3DMaharashtra%26q%3Dcivil%2520engineering%26searchFields%3D",
    "https://resumes.indeed.com/resume/e485b5745fe149eb?s=l%3DMaharashtra%26q%3DCivil%2520Engineer%26searchFields%3D",
    "https://resumes.indeed.com/resume/b0840ea0b069dc54?s=l%3DMaharashtra%26q%3DCivil%2520Engineer%26searchFields%3D",
    "https://resumes.indeed.com/resume/f29a940d7b3a460e?s=l%3DMaharashtra%26q%3DCivil%2520Engineer%26searchFields%3D",
    "https://resumes.indeed.com/resume/9735e5204e583ec3?s=l%3DMaharashtra%26q%3DCivil%2520Engineer%26searchFields%3D",
    "https://resumes.indeed.com/resume/7b873f5e0569fe2b?s=l%3DMaharashtra%26q%3DCivil%2520Engineer%26searchFields%3D"],
"Java Developer":
    ["https://resumes.indeed.com/resume/e66a00a5b491fdfd?s=l%3DMaharashtra%26q%3Djava%2520developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/138da676ff243e99?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/ca1a80fa2fbb047e?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/75b26161a7d24630?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/dec82e154fffcdfc?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/fb4ebfc71c7ba156?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/acff327e68c56c6e?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/acff327e68c56c6e?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/f5bd2240affc0e64?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/d9777bba8526b440?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/8d7c33d62a211cba?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/9bc11c5e0b434d64?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/5f79f9128c6c9f01?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/f07a170847ba3bda?s=l%3DMaharashtra%26q%3DJava%2520Developer%26searchFields%3D"],
"Business Analyst":
    ["https://resumes.indeed.com/resume/495ad979358dc20e?s=l%3DMaharashtra%26q%3Dbusiness%2520analyst%26searchFields%3D",
    "https://resumes.indeed.com/resume/52b977b5e48990a1?s=l%3DMaharashtra%26q%3DBusiness%2520Analyst%26searchFields%3D",
    "https://resumes.indeed.com/resume/b73ffd8c23445add?s=l%3DMaharashtra%26q%3DBusiness%2520Analyst%26searchFields%3D",
    "https://resumes.indeed.com/resume/7617cf7a4b7002bb?s=l%3DMaharashtra%26q%3DBusiness%2520Analyst%26searchFields%3D",
    "https://resumes.indeed.com/resume/42b8da5811921aee?s=l%3DMaharashtra%26q%3DBusiness%2520Analyst%26searchFields%3D",
    "https://resumes.indeed.com/resume/fe1951b7a478ff1a?s=l%3DMaharashtra%26q%3DBusiness%2520Analyst%26searchFields%3D"],
"SAP Developer":
    ["https://resumes.indeed.com/resume/dc1af0803fb0666d?s=l%3DMaharashtra%26q%3Dsap%2520developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/e1561e8fcb86f2dd?s=l%3DMaharashtra%26q%3DSAP%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/aa776bb1af9d3787?s=l%3DMaharashtra%26q%3DSAP%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/1ad99a5ebd10da8f?s=l%3DMaharashtra%26q%3DSAP%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/737e512d0ce13d88?s=l%3DMaharashtra%26q%3DSAP%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/140e7fe2c1cb9ebf?s=l%3DMaharashtra%26q%3DSAP%2520Developer%26searchFields%3D"],
"Automation Testing":
    ["https://resumes.indeed.com/resume/3524624cceec68e8?s=l%3DMaharashtra%26q%3Dautomation%2520testing%26searchFields%3D",
    "https://resumes.indeed.com/resume/0d611b1e4dd72ae9?s=l%3DMaharashtra%26q%3DAutomation%2520Testing%26searchFields%3D",
    "https://resumes.indeed.com/resume/f94d5a8dd26d63a8?s=l%3DMaharashtra%26q%3DAutomation%2520Testing%26searchFields%3D",
    "https://resumes.indeed.com/resume/700ded4ca9c213d3?s=l%3DMaharashtra%26q%3DAutomation%2520Testing%26searchFields%3D",
    "https://resumes.indeed.com/resume/c858bd34986ab2c1?s=l%3DMaharashtra%26q%3DAutomation%2520Testing%26searchFields%3D",
    "https://resumes.indeed.com/resume/c5ae63a75542e24e?s=l%3DMaharashtra%26q%3DAutomation%2520Testing%26searchFields%3D",
    "https://resumes.indeed.com/resume/6cc2e8a3a04cf836?s=l%3DMaharashtra%26q%3DAutomation%2520Testing%26searchFields%3D"],
"Electrical Engineering":
    ["https://resumes.indeed.com/resume/a6d5f5d050ce42c4?s=l%3DMaharashtra%26q%3Delectrical%2520engineering%26searchFields%3D",
    "https://resumes.indeed.com/resume/de5a39a651d6cee0?s=l%3DMaharashtra%26q%3DElectrical%2520Engineering%26searchFields%3D",
    "https://resumes.indeed.com/resume/aca188eb76af9859?s=l%3DMaharashtra%26q%3DElectrical%2520Engineering%26searchFields%3D",
    "https://resumes.indeed.com/resume/e3e44f7f3a249623?s=l%3DMaharashtra%26q%3DElectrical%2520Engineering%26searchFields%3D",
    "https://resumes.indeed.com/resume/c33ecfb9944a4ef5?s=l%3DMaharashtra%26q%3DElectrical%2520Engineering%26searchFields%3D"],
"Operations Manager":
    ["https://resumes.indeed.com/resume/dbe21ed7bd109e56?s=l%3DMaharashtra%26q%3Dmanager%2520operations%26searchFields%3D",
    "https://resumes.indeed.com/resume/9e55fd28c5ea7b1f?s=l%3DMaharashtra%26q%3DOperations%2520Manager%26searchFields%3D",
    "https://resumes.indeed.com/resume/4ce7442554d2152e?s=l%3DMaharashtra%26q%3DOperations%2520Manager%26searchFields%3D",
    "https://resumes.indeed.com/resume/5580c9b2a4f036b7?s=l%3DMaharashtra%26q%3DOperations%2520Manager%26searchFields%3D"],
"Python Developer":
    ["https://resumes.indeed.com/resume/7005d0da9ead0d35?s=l%3DMaharashtra%26q%3Dpython%2520developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/0683e87b6ac65306?s=l%3DMaharashtra%26q%3DPython%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/1175522899b23c80?s=l%3DMaharashtra%26q%3DPython%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/05a63c392d293dfd?s=l%3DMaharashtra%26q%3DPython%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/c53296dbb63db8bb?s=l%3DMaharashtra%26q%3DPython%2520Developer%26searchFields%3D",
    "https://resumes.indeed.com/resume/391212f0580ee5be?s=l%3DMaharashtra%26q%3DPython%2520Developer%26searchFields%3D"],
"DevOps Engineer":
    ["https://resumes.indeed.com/resume/eeee58e97c67a04c?s=l%3DMaharashtra%26q%3Ddevops%2520engineer%26searchFields%3D",
    "https://resumes.indeed.com/resume/2d8669669e59208b?s=l%3DMaharashtra%26q%3Ddevops%2520engineer%26searchFields%3D",
    "https://resumes.indeed.com/resume/cee3c6a375cc9c25?s=l%3DMaharashtra%26q%3Ddevops%2520engineer%26searchFields%3D",
    "https://resumes.indeed.com/resume/597dfc5df0a8f30a?s=l%3DMaharashtra%26q%3Ddevops%2520engineer%26rb%3Djtid%253A265%26searchFields%3D",
    "https://resumes.indeed.com/resume/0a4bdbdf4bbf6b43?s=l%3DMaharashtra%26q%3Ddevops%2520engineer%26rb%3Djtid%253A265%26searchFields%3D",
    "https://resumes.indeed.com/resume/0da2da016a815e1c?s=l%3DMaharashtra%26q%3Ddevops%2520engineer%26rb%3Djtid%253A265%26searchFields%3D",
    "https://resumes.indeed.com/resume/8ee69d0fa5c8b1da?s=l%3DMaharashtra%26q%3Ddevops%2520engineer%26rb%3Djtid%253A265%26searchFields%3D"],
"Network Security Engineer":
    ["https://resumes.indeed.com/resume/13de1f3df5f8e85d?s=l%3DMaharashtra%26q%3Dnetwork%2520security%26searchFields%3D",
    "https://resumes.indeed.com/resume/145922e907f330ea?s=l%3DMaharashtra%26q%3Dnetwork%2520security%26searchFields%3D",
    "https://resumes.indeed.com/resume/9737673a915d8e5c?s=l%3DMaharashtra%26q%3Dnetwork%2520security%26searchFields%3D",
    "https://resumes.indeed.com/resume/7ba08bc8dc78b396?s=l%3DMaharashtra%26q%3Dnetwork%2520security%26searchFields%3D",
    "https://resumes.indeed.com/resume/02a2b7cf5c63c820?s=l%3DMaharashtra%26q%3Dnetwork%2520security%26searchFields%3D"],
"PMO":
    ["https://resumes.indeed.com/resume/d61df4be001c139e?s=l%3DMaharashtra%26q%3Dpmo%2520manager%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/f63caf2d25a4ccbd?s=l%3DMaharashtra%26q%3Dpmo%2520manager%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/acac653f31adde50?s=l%3DMaharashtra%26q%3Dpmo%2520manager%26searchFields%3Djt"],
"Database":
    ["https://resumes.indeed.com/resume/824db1a58d98d1ea?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/5cd868fb4d698833?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/93407c2ef0df4d15?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/a61df877933666d3?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/073a24381efeeade?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/7e3939a1d39ca523?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/d50ddde0fea47779?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/4e147e2327fb75a1?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/35e2ac8c717e8310?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/dc6840daa62adb22?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/06ab5c6d5157cef5?s=l%3DMaharashtra%26q%3Ddatabase%2520administrator%26searchFields%3Djt"],
"Hadoop":
    ["https://resumes.indeed.com/resume/0de6aa0e8bd2bb60?s=l%3DMaharashtra%26q%3Dhadoop%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/1faea0fedc3a7005?s=l%3DMaharashtra%26q%3Dhadoop%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/6b5bfaff2063c763?s=l%3DMaharashtra%26q%3Dhadoop%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/08fa07d22b90a521?s=l%3DMaharashtra%26q%3Dhadoop%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/dbdd383ade531685?s=l%3DMaharashtra%26q%3Dhadoop%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/d718811b82fa26c9?s=l%3DMaharashtra%26q%3Dhadoop%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/fd68bea5b5557411?s=l%3DMaharashtra%26q%3Dhadoop%2520developer%26searchFields%3Djt"],
"ETL Developer":
    ["https://resumes.indeed.com/resume/5eef7e638aaad52a?s=l%3DMaharashtra%26q%3Detl%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/e5e181518ac35bfa?s=l%3DMaharashtra%26q%3Detl%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/a00d6d2c2e71705e?s=l%3DMaharashtra%26q%3Detl%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/25eefaed9efed82d?s=l%3DMaharashtra%26q%3Detl%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/c72268781359b5a3?s=l%3DMaharashtra%26q%3Detl%2520developer%26searchFields%3Djt"],
"DotNet Developer":
    ["https://resumes.indeed.com/resume/805b904bd34ac2c1?s=l%3DMaharashtra%26q%3Ddot%2520net%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/04a47edf48999faa?s=l%3DMaharashtra%26q%3Ddot%2520net%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/29baa295b5fc4051?s=l%3DMaharashtra%26q%3Ddot%2520net%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/ef408f2b5765db53?s=l%3DMaharashtra%26q%3Ddot%2520net%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/2da5b57dcbefedb1?s=l%3DMaharashtra%26q%3Ddot%2520net%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/45e8b42a60fd8c47?s=l%3DMaharashtra%26q%3Ddot%2520net%2520developer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/7445d403a0e040e1?s=l%3DMaharashtra%26q%3Ddot%2520net%2520developer%26searchFields%3Djt"],
"Blockchain":
    ["https://resumes.indeed.com/resume/bec3f890204bd008?s=l%3DMaharashtra%26q%3Dblockchain%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/c59be0d58502c3a6?s=l%3DMaharashtra%26q%3Dblockchain%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/0cb72f3195e76c05?s=l%3DMaharashtra%26q%3Dblockchain%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/868b5cf83c3f50f1?s=l%3DMaharashtra%26q%3Dblockchain%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/dae41869ade0cedf?s=l%3DMaharashtra%26q%3Dblockchain%26searchFields%3Djt"],
"Testing":
    ["https://resumes.indeed.com/resume/9c595e80bda3d5cf?s=l%3DMaharashtra%26q%3Dtesting%2520engineer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/885cc8a1f601fee9?s=l%3DMaharashtra%26q%3Dtesting%2520engineer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/ac89df10cd017bd3?s=l%3DMaharashtra%26q%3Dtesting%2520engineer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/aa1ceec6b3914dfc?s=l%3DMaharashtra%26q%3Dtesting%2520engineer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/0dc20fc9d6256822?s=l%3DMaharashtra%26q%3Dtesting%2520engineer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/7b3b05099c33786e?s=l%3DMaharashtra%26q%3Dtesting%2520engineer%26searchFields%3Djt",
    "https://resumes.indeed.com/resume/8267fa9efa87a90f?s=l%3DMaharashtra%26q%3Dtesting%2520engineer%26searchFields%3Djt"],
}
    


In [258]:
df = pd.DataFrame(columns=['Category','Resume'])
newDict = {}
for key,val in mainDict.items():
    for x in range(0,len(val)):
        response = http.request('GET', val[x])
        soup = BeautifulSoup(response.data,'html.parser',  from_encoding="utf-8")
        reqText = soup.find_all('script')[2].text
        newText  = re.findall(r"\'(.*?)\'",reqText)
        resumeText = GetRequiredResumeDetails(newText)
        str1 = ''.join(resumeText)
        df = df.append({'Category': key, 'Resume': str1}, ignore_index=True)
        #df = df.append({'Resume': str1}, ignore_index=True)
print (df)  

             Category                                             Resume
0        Data Science  Skills * Programming Languages: Python (pandas...
1        Data Science  Education Details \nMay 2013 to May 2017 B.E  ...
2        Data Science  Areas of Interest Deep Learning, Control Syste...
3        Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4        Data Science  Education Details \n MCA   YMCAUST,  Faridabad...
5        Data Science  SKILLS C Basics, IOT, Python, MATLAB, Data Sci...
6        Data Science  Skills â¢ Python â¢ Tableau â¢ Data Visuali...
7        Data Science  Education Details \n B.Tech   Rayat and Bahra ...
8        Data Science  Personal Skills â¢ Ability to quickly grasp t...
9        Data Science  Expertise â Data and Quantitative Analysis â...
10                 HR  TECHNICAL SKILLS â¢ Typewriting â¢ TORA â¢ ...
11                 HR  I.T. Skills â¢ Windows XP, Ms Office (Word, E...
12                 HR  Education Details \n BA   mu

In [259]:
df.to_csv('resume_dataset.csv', index=False)